In [1]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
from time import time
from scipy import stats


Bad key "“backend" on line 1 in
/Users/rfdanti/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution


In [18]:
trainset = pd.read_csv("full_trainset.csv")
trainset.head()

,Unnamed: 0,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,Open,Promo,Sales,SchoolHoliday,StateHoliday,competition_age_days,affectedbycompetition,affectedbypromo2,OpeningType,month
0,858297,912,2,2,3100,5,2010,0,1,2016,...,0,0,0,1,1,976,1,0,0,1
1,382600,407,0,0,5890,10,2003,1,14,2011,...,0,0,0,1,1,3380,1,1,0,1
2,137670,147,3,2,8540,8,2010,0,1,2016,...,0,0,0,1,1,884,1,0,0,1
3,897860,954,0,0,390,2,2013,1,10,2014,...,0,0,0,1,1,0,0,0,0,1
4,765983,814,3,2,15770,7,2013,0,1,2016,...,0,0,0,1,1,0,0,0,0,1


In [19]:
#Date as index
trainset.sort_values(['Store', 'Date'], ascending=[True, True], inplace=True)
trainset = trainset.reset_index(drop=True)
trainset.head()

,Unnamed: 0,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,...,Open,Promo,Sales,SchoolHoliday,StateHoliday,competition_age_days,affectedbycompetition,affectedbypromo2,OpeningType,month
0,139,1,2,0,1270,9,2008,0,1,2016,...,0,0,0,1,1,1583,1,0,0,1
1,358,1,2,0,1270,9,2008,0,1,2016,...,1,0,5530,1,0,1584,1,0,0,1
2,466,1,2,0,1270,9,2008,0,1,2016,...,1,0,4327,1,0,1585,1,0,0,1
3,550,1,2,0,1270,9,2008,0,1,2016,...,1,0,4486,1,0,1586,1,0,0,1
4,726,1,2,0,1270,9,2008,0,1,2016,...,1,0,4997,1,0,1587,1,0,0,1


In [20]:
trainset = trainset.rename(columns={"competition_age_days": "CompetitionAge", "affectedbycompetition": "AffectedByCompetition",
                        "affectedbypromo2": "AffectedByPromo2", "month": "Month"})

In [21]:
trainset = trainset.drop(columns='Unnamed: 0', axis=1)
trainset.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Open,Promo,Sales,SchoolHoliday,StateHoliday,CompetitionAge,AffectedByCompetition,AffectedByPromo2,OpeningType,Month
0,1,2,0,1270,9,2008,0,1,2016,0,...,0,0,0,1,1,1583,1,0,0,1
1,1,2,0,1270,9,2008,0,1,2016,0,...,1,0,5530,1,0,1584,1,0,0,1
2,1,2,0,1270,9,2008,0,1,2016,0,...,1,0,4327,1,0,1585,1,0,0,1
3,1,2,0,1270,9,2008,0,1,2016,0,...,1,0,4486,1,0,1586,1,0,0,1
4,1,2,0,1270,9,2008,0,1,2016,0,...,1,0,4997,1,0,1587,1,0,0,1


In [23]:
trainset["WeekOfYear"] = np.nan
trainset["DateOfMonth"] = np.nan

In [24]:
trainset['Dates'] = pd.to_datetime(trainset['Date'])
trainset['DateOfMonth'] = trainset['Dates'].dt.day

In [27]:
import datetime
trainset['WeekOfYear'] = trainset['Dates'].dt.week

In [29]:
trainset = trainset.drop(columns='Dates', axis=1)

In [34]:
#set categorical type
category_columns = ["Store", "StoreType", "Assortment", "DayOfWeek", "PromoInterval", "StateHoliday",
                   "OpeningType", "Month", "WeekOfYear", "DateOfMonth",
                    "AffectedByCompetition", "AffectedByPromo2", "StateHoliday",
                    "SchoolHoliday", "Promo", "Promo2"]
     
for column in category_columns:
    trainset[column] = trainset[column].astype('category')

In [32]:
trainset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050330 entries, 0 to 1050329
Data columns (total 27 columns):
Store                        1050330 non-null category
StoreType                    1050330 non-null category
Assortment                   1050330 non-null category
CompetitionDistance          1050330 non-null int64
CompetitionOpenSinceMonth    1050330 non-null int64
CompetitionOpenSinceYear     1050330 non-null int64
Promo2                       1050330 non-null int64
Promo2SinceWeek              1050330 non-null int64
Promo2SinceYear              1050330 non-null int64
PromoInterval                1050330 non-null category
CompetitionOpenSinceDate     1050330 non-null object
Promo2SinceDate              1050330 non-null object
Date                         1050330 non-null object
Customers                    1050330 non-null int64
DayOfWeek                    1050330 non-null category
Open                         1050330 non-null int64
Promo                        1050330 

In [35]:
trainset.to_csv("full_trainset_v2.csv", index=False)